Predition of sample's disease (tissue of origin)

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [78]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
import pickle

In [ ]:
filepath = '/Users/bogao/DataFiles/new landscape/files/singleClass_glioblastoma.pkl'

In [28]:
filepath = '/Users/bogao/DataFiles/new landscape/files/singleClass_melanoma.pkl'

In [47]:
with open(filepath, 'rb') as fi:
    data = pickle.load(fi)

In [48]:
size = int(data.shape[0]/2)
labels = [0]*size + [1]*size

In [54]:
pos_size = 1030
neg_size = int(data.shape[0]) - pos_size
labels = [0]*pos_size + [1]*neg_size

In [49]:
data.shape[0]

42612

In [79]:
model = RandomForestClassifier(n_estimators=1000)

In [56]:

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.30, random_state=5)

In [57]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [58]:
y_pred = model.predict(x_test)

In [59]:
metrics.accuracy_score(y_test, y_pred)

0.9775500625782227

In [60]:
metrics.recall_score(y_test, y_pred)

0.9998395507420779

In [61]:
metrics.precision_score(y_test, y_pred)

0.9776435519297144

In [74]:
metrics.f1_score(y_test, y_pred)

0.9886169833022647

In [72]:
scores = cross_val_score(model, x_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True), n_jobs=-1)

In [73]:
scores

array([0.97737557, 0.97737177, 0.97937972, 0.97837385, 0.97787091])

In [75]:
matfile = '/Users/bogao/DataFiles/new landscape/data/selected_gene_mat.pkl'
labelfile = '/Users/bogao/DataFiles/new landscape/data/all_bands_label.pkl'
dlabelfile = '/Users/bogao/DataFiles/new landscape/data/all_bands_disease_label.pkl'

In [76]:
with open(matfile, 'rb') as fmat, open(labelfile, 'rb') as flab, open(dlabelfile, 'rb') as fdl:
        data = pickle.load(fmat)
        data = np.array(data)
        labels = pickle.load(flab)
        d_labels = pickle.load(fdl)

In [173]:
d = data[(d_labels !='Others') & (d_labels !='None')]
l = labels[(d_labels !='Others') & (d_labels !='None')]
t = d_labels[(d_labels !='Others') & (d_labels !='None')]

In [139]:
d.shape

(22655, 3029)

In [140]:
t.shape

(22655,)

In [99]:
x_train, x_test, y_train, y_test = train_test_split(d, l, test_size=0.30, shuffle=True, stratify=l)

In [101]:
mul_model = OneVsRestClassifier(model, n_jobs=-1)

In [102]:
mul_model.fit(x_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=1000,
                                                     n_jobs=None,
                                             

In [103]:
pred = mul_model.predict(x_test)

In [83]:
pred = OneVsRestClassifier(model, n_jobs=-1).fit(x_train, y_train).predict(x_test)

In [100]:
np.unique(l)

array(['Brain', 'Breast', 'Colon', 'Kidney', 'Liver', 'Lung', 'Ovary',
       'Prostate', 'Skin', 'Stomach'], dtype=object)

In [106]:
metrics.accuracy_score(y_test, pred)

0.6963710857477319

In [93]:
metrics.recall_score(y_test, pred, average=None)

array([0.51364176, 0.60170849, 0.50181818, 0.44230769, 0.10891089,
       0.3697479 , 0.94179278, 0.29467085, 0.1448468 , 0.29110512,
       0.05882353])

In [94]:
metrics.precision_score(y_test, pred, average=None)

array([0.88367347, 0.73276983, 0.75409836, 0.94358974, 0.81481481,
       0.74324324, 0.61567732, 0.79661017, 0.85245902, 0.90756303,
       0.95833333])

In [104]:
mul_model.classes_

array(['Brain', 'Breast', 'Colon', 'Kidney', 'Liver', 'Lung', 'Ovary',
       'Prostate', 'Skin', 'Stomach'], dtype='<U8')

In [174]:
names, counts = np.unique(t, return_counts=True)

In [175]:
classes = names[counts>100]

In [176]:
d = d[np.isin(t, classes)]
t = t[np.isin(t, classes)]

In [178]:
x_train, x_test, y_train, y_test = train_test_split(d, t, test_size=0.30, shuffle=True, stratify=t)

In [179]:
mul_model.fit(x_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=1000,
                                                     n_jobs=None,
                                             

In [180]:
pred = mul_model.predict(x_test)

In [181]:
metrics.accuracy_score(y_test, pred)

0.5861467744491996

In [182]:
metrics.recall_score(y_test, pred, average=None)

array([0.10902256, 0.06382979, 0.33783784, 0.19254658, 0.72961058,
       0.        , 0.21621622, 0.        , 0.73867596, 0.09278351,
       0.35294118, 0.        , 0.89228958, 0.13333333, 0.48867314,
       0.39622642, 0.        , 0.04081633, 0.        , 0.01851852,
       0.71659919, 0.39393939, 0.12903226, 0.03947368, 0.44736842])

In [183]:
metrics.precision_score(y_test, pred, average=None)

/Users/bogao/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([0.90625   , 1.        , 0.55732484, 0.484375  , 0.51691827,
       0.        , 0.77419355, 0.        , 0.75714286, 0.9       ,
       0.71328671, 0.        , 0.56630557, 0.5       , 0.72596154,
       0.95454545, 0.        , 0.33333333, 0.        , 0.5       ,
       0.75319149, 0.5       , 0.33333333, 1.        , 0.89473684])

In [184]:
mul_model.classes_

array(['80103', '80413', '80463', '80703', '81403', '81443', '81703',
       '82553', '83103', '83123', '84413', '84803', '85003', '85203',
       '87203', '89363', '93803', '93823', '94003', '94013', '94403',
       '94503', '94513', 'C22', 'C61'], dtype='<U5')

In [185]:
metrics.f1_score(y_test, pred, average=None)

/Users/bogao/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([0.19463087, 0.12      , 0.42067308, 0.27555556, 0.60511883,
       0.        , 0.33802817, 0.        , 0.74779541, 0.1682243 ,
       0.47222222, 0.        , 0.6928702 , 0.21052632, 0.58413926,
       0.56      , 0.        , 0.07272727, 0.        , 0.03571429,
       0.73443983, 0.44067797, 0.18604651, 0.07594937, 0.59649123])